## Step 0 - Installing Librariess

In [2]:
!pip install -q --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install -q sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install -q --no-deps unsloth

In [3]:
import os
os.environ["UNSLOTH_COMPILE_DISABLE"] = "1" 


## Step 1 - Choose a Base Model

In [4]:
from unsloth import FastModel
import torch
from transformers import AutoConfig # Import AutoConfig

# Choose any model of your choosing, based on your GPU RAM
#default = "md-nishat-008/TigerLLM-1B-it"
# gemma3_1b = "google/gemma-3-1b-it"
# gemma3_4b = "google/gemma-3-4b-it"
# gemma3_12b = "google/gemma-3-12b-it"
# gemma3_27b = "google/gemma-3-27b-it"
default="meta-llama/Meta-Llama-3-8B-Instruct"
# Load model with the corrected config
model, tokenizer = FastModel.from_pretrained(
    model_name = default,
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    use_flash_attention_2 = False,       # 🚫 disables FlashAttention 2
    #use_torch_compile = False, 
    # token = "hf_...", # In case you need it
)


# Note - You will find your Hugging Face Access Token in your Hugging Face Account.
# This will be necessary for Gemma & LLaMA models.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-08-18 15:06:46.978654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755529607.236504      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755529607.308988      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.7: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Step 2 - Play with the Base Model

In [5]:
# from unsloth.chat_templates import get_chat_template

# # Attach the TigerLLM & Gemma-3 chat template to the tokenizer
# tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# # Compose the conversation
# bangla_q = (
#     "মৌলিক সংখ্যা বের করার জন্য একটি পাইথন ফাংশন লিখুন"
# )


# messages = [
#     {
#         "role": "user",
#         "content": bangla_q,
#     }
# ]

# # Render the chat template to plain text
# chat_text = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,  # must be True for generation
#     tokenize=False               # return a plain string
# )

# # Tokenize and generate
# inputs = tokenizer(chat_text, return_tensors="pt").to("cuda")

# outputs = model.generate(
#     **inputs,
#     use_cache=False,
#     max_new_tokens=256,           # increase for longer outputs
#     temperature=0.3,
#     top_p=0.95,
#     top_k=64,
# )

# print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

user

মৌলিক সংখ্যা বের করার জন্য একটি পাইথন ফাংশন লিখুনassistant

A Python function to find the prime numbers!

Here's a simple function that uses a helper function to check if a number is prime:
```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def find_primes(n):
    primes = []
    for i in range(2, n + 1):
        if is_prime(i):
            primes.append(i)
    return primes
```
Here's an explanation of the code:

1. The `is_prime` function takes an integer `n` as input and returns `True` if it's prime, and `False` otherwise. It does this by checking if `n` is less than or equal to 1 (in which case it's not prime), or if it has any divisors other than 1 and itself (by iterating from 2 to the square root of `n` and checking if `n` is divisible by any of these numbers).
2. The `find_primes` function takes an integer `n` as input and returns a list of all prime numb

In [17]:
from unsloth.chat_templates import get_chat_template
import torch

# Attach Llama-3 chat template to the tokenizer
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Ensure a pad token is set (Llama-3 typically uses EOS as PAD)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

messages = [
    {"role": "system", "content": "You are a Python tutor who always writes correct and runnable code."},
    {"role": "user", "content": "মৌলিক সংখ্যা বের করার জন্য একটি পাইথন ফাংশন লিখুন।"}
]


device = getattr(model, "device", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Build model inputs directly from the chat template
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,   # important for generation
    return_tensors="pt"           # get tensors directly
)

# Move to the same device as the model
input_ids = input_ids.to(device)

# Explicit attention mask to avoid PAD==EOS ambiguity
attention_mask = (input_ids != tokenizer.pad_token_id).long()

# Generate (enable sampling so temperature/top_p/top_k are used)
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.3,
    top_p=0.95,
    top_k=64,
    use_cache=True,
)

# Decode only the completion (exclude the prompt)
prompt_len = input_ids.shape[-1]
completion_ids = outputs[0, prompt_len:]
text = tokenizer.decode(completion_ids, skip_special_tokens=True).strip()

print(text)


A simple()


## Step 3 - Set the LoRA Adapters

In [6]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 32,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


## Step 4 - Data Prep

### Chat Template

In [7]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
)

### Use Your Own Instruction Dataset
### You will find a sample dataset here - https://noshinulfat.github.io/blp25_code_generation_task/#/task-announcement

In [8]:
## We will use a very small sample dataset
from datasets import load_dataset

# Load the local CSV (will appear as the single default “train” split)
dataset = load_dataset(
    "csv",
    data_files="/kaggle/input/task-2/trial.csv",   # path relative to your current directory
)["train"]                               # pull the one split that load_dataset creates

print(len(dataset), "rows loaded.")


Generating train split: 0 examples [00:00, ? examples/s]

74 rows loaded.


### Map

In [9]:
def to_finetome(example):
    return {
        "conversations": [
            {"role": "user",  "content": example["instruction"]},
            {"role": "model", "content": example["response"]},   # ← key change
        ]
    }

dataset = dataset.map(
    to_finetome,
    num_proc=4,
    remove_columns=["instruction", "response"]
)

Map (num_proc=4):   0%|          | 0/74 [00:00<?, ? examples/s]

### Check the First Instance

In [10]:
dataset[0]

{'id': 1,
 'test_list': '"[\'assert smallest_multiple(13)==360360\', \'assert smallest_multiple(2)==2\', \'assert smallest_multiple(1)==1\']"',
 'conversations': [{'content': 'প্রথম n সংখ্যার ক্ষুদ্রতম গুণিতক খুঁজে বের করার জন্য একটি ফাংশন লিখুন।',
   'role': 'user'},
  {'content': 'def smallest_multiple(n):\r\n    if (n<=2):\r\n      return n\r\n    i = n * 2\r\n    factors = [number  for number in range(n, 1, -1) if number * 2 > n]\r\n    while True:\r\n        for a in factors:\r\n            if i % a != 0:\r\n                i += n\r\n                break\r\n            if (a == factors[-1] and i % a == 0):\r\n                return i',
   'role': 'model'}]}

### Formatting

In [11]:
def formatting_prompts_func(batch):
    texts = []
    for convo in batch["conversations"]:
        try:
            serialized = tokenizer.apply_chat_template(
                convo,
                tokenize=False,
                add_generation_prompt=False,
            ).removeprefix("<bos>")
        except Exception:          # catches TemplateError, etc.
            serialized = ""        # placeholder keeps list length in sync
        texts.append(serialized)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/74 [00:00<?, ? examples/s]

### Check the First Instance Again

In [12]:
dataset[0]["text"]

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nপ্রথম n সংখ্যার ক্ষুদ্রতম গুণিতক খুঁজে বের করার জন্য একটি ফাংশন লিখুন।<|eot_id|><|start_header_id|>model<|end_header_id|>\n\ndef smallest_multiple(n):\r\n    if (n<=2):\r\n      return n\r\n    i = n * 2\r\n    factors = [number  for number in range(n, 1, -1) if number * 2 > n]\r\n    while True:\r\n        for a in factors:\r\n            if i % a != 0:\r\n                i += n\r\n                break\r\n            if (a == factors[-1] and i % a == 0):\r\n                return i<|eot_id|>'

## Step 5 - Finetuning Config
### Adjust as you see fit

In [13]:
from trl import SFTTrainer, SFTConfig
import math

# --- constants ----------------------------------------------------------
DATASET_SIZE                = 74
PER_DEV_BATCH               = 16
GRAD_ACC_STEPS              = 4
EPOCHS                      = 2

# derived values (no longer passed as kwargs)
EFFECTIVE_BATCH             = PER_DEV_BATCH * GRAD_ACC_STEPS
STEPS_PER_EPOCH             = math.ceil(DATASET_SIZE / EFFECTIVE_BATCH)
MAX_STEPS                   = EPOCHS * STEPS_PER_EPOCH

cfg = SFTConfig(
    dataset_text_field          = "text",
    # ── memory / speed knobs ────────────────────────────────────────────
    packing                     = False,
    per_device_train_batch_size = PER_DEV_BATCH,
    gradient_accumulation_steps = GRAD_ACC_STEPS,
    gradient_checkpointing      = True,
    bf16                        = False,
    fp16                        = True,
    optim                       = "adamw_8bit",

    # ── schedule / optimisation ────────────────────────────────────────
    num_train_epochs            = EPOCHS,       # renamed
    max_steps                   = MAX_STEPS,    # keeps the same target
    warmup_steps                = 10,
    lr_scheduler_type           = "cosine",
    learning_rate               = 1e-4,
    weight_decay                = 0.01,

    # ── misc ───────────────────────────────────────────────────────────
    logging_steps               = 1,  ## Update this
    dataset_num_proc            = 8,
    seed                        = 3407,
    report_to                   = "none",
)

trainer = SFTTrainer(
    model          = model,        # load with attn_implementation="flash_attention_2"
    tokenizer      = tokenizer,
    train_dataset  = dataset,
    args           = cfg,
)


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/74 [00:00<?, ? examples/s]

### Masking

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part    = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=4):   0%|          | 0/74 [00:00<?, ? examples/s]

ZeroDivisionError: Unsloth: All labels in your dataset are -100. Training losses will be all 0.
For example, are you sure you used `train_on_responses_only` correctly?
Or did you mask our tokens incorrectly? Maybe this is intended?
Maybe you're using a Llama chat template on a non Llama model for example?

### Verify the Masking

In [ ]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

### Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[0]["labels"]]).replace(tokenizer.pad_token, " ")

## Step 6 - Let's Finetune 🔥🔥

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

### Start Finetuning....... 🔥🔥

In [ ]:
trainer_stats = trainer.train()

## Step 7 - Let's Use Our New Finetuned Model

In [ ]:
from unsloth.chat_templates import get_chat_template

# Attach the Gemma-3 chat template to the tokenizer
tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Compose the conversation
bangla_q = (
    "মৌলিক সংখ্যা বের করার জন্য একটি পাইথন ফাংশন লিখুন"
)

messages = [
    {
        "role": "user",
        "content": bangla_q,
    }
]

# Render the chat template to plain text
chat_text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,  # must be True for generation
    tokenize=False               # return a plain string
)

# Tokenize and generate
inputs = tokenizer(chat_text, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    use_cache=False,
    max_new_tokens=1012,           # increase for longer outputs
    temperature=0.3,
    top_p=0.95,
    top_k=64,
)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


## Step 8 - Use New Model on Dev Set

In [ ]:
import pandas as pd
import torch
from tqdm.auto import tqdm
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(tokenizer, chat_template="llama-3")

# Load prompts
df = pd.read_csv("/kaggle/input/task-2/dev_v2.csv")  # expects columns: id, instruction
assert {"id", "instruction"}.issubset(df.columns), "CSV must have columns: id, instruction"

responses = []
for prompt in tqdm(df["instruction"], desc="Generating"):
    chat_text = tokenizer.apply_chat_template(
        [{"role": "user", "content": str(prompt)}],
        add_generation_prompt=True,
        tokenize=False,
    )
    inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            use_cache=False,
            max_new_tokens=256,
            temperature=0.3,
            top_p=0.95,
            top_k=64,
        )

    gen_ids  = out[0][inputs["input_ids"].shape[1]:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
    responses.append(gen_text)

# Save ONLY id + response as JSON
out_df = pd.DataFrame({"id": df["id"], "response": responses})
out_df.to_json("submission.json", orient="records", force_ascii=False, indent=2)
print(f"✅ Wrote submission.json with {len(out_df)} rows (id, response).")


## Step 9 - Preparing Submission File

In [ ]:
# import json, os, re, zipfile

# SUB_PATH = "submission.json"

# def file_format_check(path: str) -> bool:
#     # name + extension
#     if os.path.basename(path) != "submission.json":
#         print("Error: File name must be exactly 'submission.json'")
#         return False
#     if not path.lower().endswith(".json"):
#         print("Error: File must have .json extension")
#         return False

#     # must be valid JSON (not JSONL) and root must be a list
#     try:
#         with open(path, "r", encoding="utf-8") as f:
#             data = json.load(f)
#     except json.JSONDecodeError as e:
#         print(f"Error: Invalid JSON format - {e}")
#         print("Note: The file must be in proper JSON format (not JSONL)")
#         return False

#     if not isinstance(data, list):
#         print("Error: The root element should be a list of objects")
#         return False

#     # each item: dict with ONLY keys {'id','response'}; id=int; response=str
#     for idx, item in enumerate(data):
#         if not isinstance(item, dict):
#             print(f"Error: Item at index {idx} is not a dictionary")
#             return False
#         keys = set(item.keys())
#         if keys != {"id", "response"}:
#             print(f"Error: Item at index {idx} must contain only keys 'id' and 'response', found: {keys}")
#             return False
#         if not isinstance(item["id"], int):
#             print(f"Error: 'id' field at index {idx} must be an integer")
#             return False
#         if not isinstance(item["response"], str):
#             print(f"Error: 'response' field at index {idx} must be a string")
#             return False

#     print("Format check passed successfully!")
#     return True

# # ---------- Load, compute per-item validity, blank invalids, save, zip ----------
# # Load JSON list
# with open(SUB_PATH, "r", encoding="utf-8") as f:
#     data = json.load(f)

# n = len(data)
# fence_pat = re.compile(r"^```python[\s\S]*```$", re.MULTILINE)

# valid_format = []
# valid_fence  = []
# valid_both   = []

# # Per-item validation mirrors file checker semantics
# def item_format_ok(item):
#     return (
#         isinstance(item, dict)
#         and set(item.keys()) == {"id", "response"}
#         and isinstance(item["id"], int)
#         and isinstance(item["response"], str)
#     )

# for item in data:
#     vfmt = item_format_ok(item)
#     vf   = bool(fence_pat.match(item["response"])) if vfmt else False
#     valid_format.append(vfmt)
#     valid_fence.append(vf)
#     valid_both.append(vfmt and vf)

# # Report stats
# nf = sum(valid_fence)
# nm = sum(valid_format)
# nb = sum(valid_both)
# den = max(n, 1)
# print(f"Fencing valid: {nf}/{n} ({nf*100.0/den:.1f}%)")
# print(f"Format valid:  {nm}/{n} ({nm*100.0/den:.1f}%)")
# print(f"Both valid:    {nb}/{n} ({nb*100.0/den:.1f}%)")

# # Strict policy: blank responses that fail ANY check
# for i, ok in enumerate(valid_both):
#     if not ok and isinstance(data[i], dict) and "response" in data[i]:
#         data[i]["response"] = ""

# # Overwrite submission.json (id+response only)
# with open(SUB_PATH, "w", encoding="utf-8") as f:
#     json.dump(
#         [{"id": item["id"], "response": item["response"]} for item in data],
#         f, ensure_ascii=False, indent=2
#     )
# print("✅ Updated submission.json after checks (invalid responses blanked).")

# # Final file-level check (should pass)
# _ = file_format_check(SUB_PATH)

# # Zip as submission.zip (Jupyter-friendly, no shell commands)
# with zipfile.ZipFile("submission.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
#     zf.write(SUB_PATH)
# print("📦 Created submission.zip containing submission.json.")


# Submit the submission.zip file in CodaBench

### Save the NEW model...if it's good :)

In [ ]:
# model.save_pretrained("New_Model")  # Local saving
# tokenizer.save_pretrained("New_Model")